In [28]:
import os
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta
import re


BASE_URL = 'https://thenewsmill.com/'
country = 'bangladesh'
initial_url = f'{BASE_URL}?s={country}'

#ADDITIONAL NEWS LINKS ARE LOADED BY JUST SCROLLING AND WAITING. NO BUTTONS NEED TO BE PRESSED. 

#SO WE SCROLL 

driver = webdriver.Chrome()
driver.get(initial_url)

time.sleep(5)

top_element = driver.find_element(By.ID, 'main')
last_element = driver.find_element(By.ID, 'footer-widgets')

number_of_scrolls = 2

while number_of_scrolls > 0:

    driver.execute_script("arguments[0].scrollIntoView();", last_element)

    time.sleep(10)
    
    number_of_scrolls -= 1
    
    driver.execute_script("arguments[0].scrollIntoView();", top_element)
    
    time.sleep(5)

soup = BeautifulSoup(driver.page_source, 'html.parser')

main_div = soup.find('div', class_ = 'generate-columns-container')

all_news_links = []

all_articles = main_div.find_all('article')

for each_article in all_articles:
    
    all_news_links.append(each_article.find('a').get('href'))

print(f'Number of article links collected: {len(all_news_links)}')

driver.quit()

Number of article links collected: 60


In [1]:
import os
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta
import re


BASE_URL = 'https://thenewsmill.com/'
country = 'bangladesh'
initial_url = f'{BASE_URL}?s={country}'

#ADDITIONAL NEWS LINKS ARE LOADED BY JUST SCROLLING AND WAITING. NO BUTTONS NEED TO BE PRESSED. 

#SO WE SCROLL FOR A PARTICULAR NUMBER OF TIMES TO LOAD NEWS.

driver = webdriver.Chrome()
driver.get(initial_url)

time.sleep(5)

top_element = driver.find_element(By.ID, 'main')
last_element = driver.find_element(By.ID, 'footer-widgets')

#COLLECTING ALL NEWS URLS

number_of_scrolls = 2

while number_of_scrolls > 0:

    driver.execute_script("arguments[0].scrollIntoView();", last_element)

    time.sleep(10)
    
    number_of_scrolls -= 1
    
    driver.execute_script("arguments[0].scrollIntoView();", top_element)
    
    time.sleep(5)

soup = BeautifulSoup(driver.page_source, 'html.parser')

main_div = soup.find('div', class_ = 'generate-columns-container')

all_news_links = []

all_articles = main_div.find_all('article')

for each_article in all_articles:
    
    all_news_links.append(each_article.find('a').get('href'))

print(f'Number of article links collected: {len(all_news_links)}')

driver.quit()

#SCRAPING ALL LINKS
counter = 0
data_list = []

for url in all_news_links[:20]:

        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        #TITLE
        main_tag = soup.find('h1', class_ = 'gb-headline gb-headline-f593ba8c gb-headline-text')
        title = main_tag.text if main_tag else 'Title not found'
        title_translation = 'None'

        #AUTHOR
        author_tag = soup.find('p', class_ = 'gb-headline gb-headline-d90aa5ca gb-headline-text')
        author = author_tag.find('a').text if author_tag else 'Author not found'

        #DATE

        date_info = soup.find('time', class_ = 'entry-date published')

        if date_info:

            date_data = date_info.get('datetime')

            only_date = date_data.split('T')[0]
            only_time = date_data.split('T')[1]
            time = f"{only_time.split(':')[0]}:{only_time.split(':')[1]}"
            cleaned_date = f"{only_date},{time}"

            source_localtime = datetime.strptime(cleaned_date, "%Y-%m-%d,%H:%M")
            bangladesh_localtime = source_localtime + timedelta(minutes=30)

        else:
            date_data = 'Date data not found'

        content = []

        main_container = soup.find('div', class_ = 'gb-container gb-container-1cdc7c96')

        if main_container:

            all_paras = main_container.find_all('p')

            for each_para in all_paras:

                content.append(each_para.text)

            full_content = ''.join(content)
            full_content = full_content.strip()
        
        else:
            
            full_content = 'Content not found'

        content_translation = 'None'

        #NO CONTENT SUMMARIES IN THIS WEBSITE
        content_summary = 'None'
        summary_translation = 'None'

        data_dict = {
                "url": url,
                "title": title,
                "content": full_content,
                "content_summary": content_summary,
                "title_translation":title_translation,
                "content_translation":content_translation,
                "summary translation":summary_translation,
                "author": author,
                "country": country,
                'source_localtime': source_localtime,
                'bangladesh_localtime': bangladesh_localtime

            }

        counter+=1


        if (full_content != "Content not found" or title != 'Title not found'):

                    if data_dict not in data_list:
                            # Adding to data list
                            data_list.append(data_dict)
#                             print(f'Link {counter} added')
        else:
                    print(f'Link {counter}')
                    print('Skipped due to missing info.')
                    
                    
df = pd.DataFrame(data_list)
df.head()

csv_filename = f"{country}_The_News_Mill.csv"

# Checking if the CSV file already exists
if os.path.exists(csv_filename):
    existing_df = pd.read_csv(csv_filename)
    # Merging new and existing dataframe
    df = pd.concat([existing_df, pd.DataFrame(data_list)], ignore_index=True)
    df["bangladesh_localtime"] = pd.to_datetime(df["bangladesh_localtime"])  # Converting the "date" column to datetime
    df = df.drop_duplicates(subset=["title"], keep="first")
    df = df.sort_values(by="date", ascending=False)  # Sorting the date
    df = df.reset_index(drop=True)
    df.to_csv(csv_filename, index=False)  
else:
    # If csv file does not exist, then we create a new CSV file with the scraped data
    df = pd.DataFrame(data_list)
    df = df.sort_values(by="bangladesh_localtime", ascending=False)
    df = df.drop_duplicates(subset=["title"], keep="first")
    df = df.reset_index(drop=True)
    df.to_csv(csv_filename, index=False)


Number of article links collected: 60


,url,title,content,content_summary,title_translation,content_translation,summary translation,author,country,source_localtime,bangladesh_localtime
0,https://thenewsmill.com/2024/03/it-will-be-a-g...,“It will be a good series”: Shanto opens up on...,Ahead of Bangladesh’s three-match T20I series ...,None,None,None,None,ANI,bangladesh,2024-03-04 02:07:00,2024-03-04 02:37:00
1,https://thenewsmill.com/2024/03/more-unemploye...,“More unemployed youths in India than Banglade...,During the Madhya Pradesh leg of the Bharat Jo...,None,None,None,None,ANI,bangladesh,2024-03-03 13:05:00,2024-03-03 13:35:00
2,https://thenewsmill.com/2024/02/sri-lanka-anno...,Sri Lanka announce T20I squad against Banglade...,Sri Lanka Cricket (SLC) on Wednesday announced...,None,None,None,None,ANI,bangladesh,2024-02-29 00:02:00,2024-02-29 00:32:00
3,https://thenewsmill.com/2024/02/bangladesh-app...,"Bangladesh appoints new batting, bowling coach...",The Bangladesh Cricket Board (BCB) appointed D...,None,None,None,None,ANI,bangladesh,2024-02-28 12:03:00,2024-02-28 12:33:00
4,https://thenewsmill.com/2024/02/mea-additional...,MEA Additional Secy interacts with 100-member ...,"P Kumaran, the Additional Secretary at the Min...",None,None,None,None,ANI,bangladesh,2024-02-27 20:01:00,2024-02-27 20:31:00
